### *Implementing AI-Powered Chatbot in B2C Customer Experience Using Natural Language Processing*

##### Import Libraries

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

##### Pre-process Data

In [2]:
# Tokenize intents

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        # Add documents in the corpus
        documents.append((w, intent['tag']))

        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# Print tokenized words
print(words)

["'m", "'s", '.', 'a', 'about', 'accept', 'agent', 'am', 'an', 'any', 'anyone', 'are', 'available', 'be', 'become', 'but', 'buy', 'bye', 'call', 'can', 'cancel', 'cancelled', 'card', 'cash', 'common', 'completed', 'confirmation', 'confirmed', 'contact', 'credit', 'credited', 'customer', 'day', 'deducted', 'delay', 'delayed', 'delivered', 'delivery', 'did', 'do', 'doe', 'doing', 'even', 'for', 'from', 'fund', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helpline', 'hey', 'hi', 'how', 'i', 'im', 'is', 'it', 'item', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'method', 'money', 'more', 'my', "n't", 'need', 'no', 'not', 'number', 'of', 'offer', 'okay', 'order', 'pay', 'payment', 'please', 'processed', 'reason', 'receive', 'received', 'refund', 'refunded', 'return', 'returned', 'say', 'see', 'sell', 'seller', 'service', 'shipping', 'should', 'status', 'successfully', 'support', 'take', 'talk', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'thnks', 'tho

In [5]:
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# Assuming 'words' contains the tokenized words
lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

# Print lemmatized words
print(lemmatized_words)

["'m", "'s", '.', 'a', 'about', 'accept', 'agent', 'am', 'an', 'any', 'anyone', 'are', 'available', 'be', 'become', 'but', 'buy', 'bye', 'call', 'can', 'cancel', 'cancelled', 'card', 'cash', 'common', 'completed', 'confirmation', 'confirmed', 'contact', 'credit', 'credited', 'customer', 'day', 'deducted', 'delay', 'delayed', 'delivered', 'delivery', 'did', 'do', 'doe', 'doing', 'even', 'for', 'from', 'fund', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helpline', 'hey', 'hi', 'how', 'i', 'im', 'is', 'it', 'item', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'method', 'money', 'more', 'my', "n't", 'need', 'no', 'not', 'number', 'of', 'offer', 'okay', 'order', 'pay', 'payment', 'please', 'processed', 'reason', 'receive', 'received', 'refund', 'refunded', 'return', 'returned', 'say', 'see', 'sell', 'seller', 'service', 'shipping', 'should', 'status', 'successfully', 'support', 'take', 'talk', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'thnks', 'tho

In [3]:
# Lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# Sort classes
classes = sorted(list(set(classes)))

# Documents = combination between patterns and intents
print (len(documents), "documents")

# Classes = intents
print (len(classes), "classes", classes)

# Words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

94 documents
20 classes ['cancel', 'confirm', 'delay', 'delivery', 'goodbye', 'greeting', 'items', 'more', 'order status', 'payments', 'personal', 'reasons', 'refund', 'refund status', 'return', 'thanks', 'track', 'use voucher', 'user response', 'withdraw refunds']
131 unique lemmatized words ["'m", "'s", '.', 'a', 'about', 'accept', 'agent', 'am', 'an', 'any', 'anyone', 'are', 'available', 'be', 'become', 'but', 'buy', 'bye', 'call', 'can', 'cancel', 'cancelled', 'card', 'cash', 'common', 'completed', 'confirmation', 'confirmed', 'contact', 'credit', 'credited', 'customer', 'day', 'deducted', 'delay', 'delayed', 'delivered', 'delivery', 'did', 'do', 'doe', 'doing', 'even', 'for', 'from', 'fund', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helpline', 'hey', 'hi', 'how', 'i', 'im', 'is', 'it', 'item', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'method', 'money', 'more', 'my', "n't", 'need', 'no', 'not', 'number', 'of', 'offer', 'okay', 'order', 'pay',

##### Create training and testing data

In [6]:
# Create our training data
training = []
output_empty = [0] * len(classes)

# Create bag of words for each sentence
for doc in documents:
    bag = [1 if w in doc[0] else 0 for w in words]
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Separate bag-of-words vectors and output rows
train_x = [entry[0] for entry in training]
train_y = [entry[1] for entry in training]

# Convert lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

print("Training Data Created Successfully.")

Training Data Created Successfully.


In [7]:
# Create model with 3 layers. First layer 128 neurons, second layer 64 neurons
# and 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('hausa_chatbot_model.h5', hist)

print("Model Created Successfully")

Epoch 1/200
19/19 [==============================] - 1s 3ms/step - loss: 3.0354 - accuracy: 0.0426 
Epoch 2/200
19/19 [==============================] - 0s 3ms/step - loss: 2.9303 - accuracy: 0.1064
Epoch 3/200
19/19 [==============================] - 0s 3ms/step - loss: 2.8971 - accuracy: 0.1277
Epoch 4/200
19/19 [==============================] - 0s 2ms/step - loss: 2.7788 - accuracy: 0.1489
Epoch 5/200
19/19 [==============================] - 0s 3ms/step - loss: 2.7200 - accuracy: 0.1809
Epoch 6/200
19/19 [==============================] - 0s 3ms/step - loss: 2.5532 - accuracy: 0.2021
Epoch 7/200
19/19 [==============================] - 0s 4ms/step - loss: 2.5304 - accuracy: 0.2234
Epoch 8/200
19/19 [==============================] - 0s 5ms/step - loss: 2.3977 - accuracy: 0.2660
Epoch 9/200
19/19 [==============================] - 0s 3ms/step - loss: 2.2943 - accuracy: 0.3085
Epoch 10/200
19/19 [==============================] - 0s 4ms/step - loss: 2.1698 - accuracy: 0.3298
Epoch 11

19/19 [==============================] - 0s 4ms/step - loss: 0.3189 - accuracy: 0.8936
Epoch 84/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3377 - accuracy: 0.8511
Epoch 85/200
19/19 [==============================] - 0s 3ms/step - loss: 0.3493 - accuracy: 0.8830
Epoch 86/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2691 - accuracy: 0.8936
Epoch 87/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2659 - accuracy: 0.9043
Epoch 88/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3017 - accuracy: 0.8830
Epoch 89/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3034 - accuracy: 0.8830
Epoch 90/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2734 - accuracy: 0.8723
Epoch 91/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3076 - accuracy: 0.8936
Epoch 92/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3282 - accuracy: 0.8723
Epoch 93/200


19/19 [==============================] - 0s 4ms/step - loss: 0.1919 - accuracy: 0.9043
Epoch 165/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2544 - accuracy: 0.8936
Epoch 166/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2575 - accuracy: 0.9149
Epoch 167/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2832 - accuracy: 0.8830
Epoch 168/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2331 - accuracy: 0.8830
Epoch 169/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1953 - accuracy: 0.9043
Epoch 170/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2008 - accuracy: 0.9468
Epoch 171/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2123 - accuracy: 0.9149
Epoch 172/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2505 - accuracy: 0.8936
Epoch 173/200
19/19 [==============================] - 0s 2ms/step - loss: 0.2509 - accuracy: 0.9149
Epoc

C:\Users\olade\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               16896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 20)                1300      
                                                                 
Total params: 26452 (103.33 KB)
Trainable params: 26452 (103.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
from keras.models import load_model
model = load_model('hausa_chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [10]:
import random
from googletrans import Translator

def translate_to_hausa(text):
    # Create a translator object
    translator = Translator()
    
    # Translate the text to Hausa
    translated = translator.translate(text, src='en', dest='ha')
    
    return translated.text

In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [12]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [13]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [14]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            
            # Translate the response to Hausa
            hausa_translation = translate_to_hausa(result)
            break
    return hausa_translation

In [15]:
def chatbot_response(text):
    ints = predict_class(text, model)  # Assuming you have the predict_class function implemented
    res = getResponse(ints, intents)   # Assuming you have the intents data loaded and the getResponse function implemented
    
    # Translate the response to Hausa
    hausa_translation = translate_to_hausa(res)
    
    return hausa_translation

#### GUI Interface

In [53]:
import tkinter
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#c5f0e3"
TEXT_COLOR = "#000000"

# BG_GRAY = "#ABB2B9"
# BG_COLOR = "#1c172a"
# TEXT_COLOR = "#ffffff"


FONT = "Helvetica 8"
FONT_BOLD = "Helvetica 13 bold"


def send(event = None):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#000000", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Sahad Bot: " + res + '\n\n')

        ChatLog.config(state = DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("E-Commerce Chatbot")
base.resizable(width=FALSE, height=FALSE)
base.configure(width=500, height=500, bg=BG_COLOR)


#Create Chat window
ChatLog = Text(base, bd=0, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT_BOLD)
ChatLog.config(state=DISABLED)

head_label = Label(base, bg=BG_COLOR, fg=TEXT_COLOR, text = "E-Commerce Chatbot for Sahad Electronics Store", font=FONT_BOLD, pady=10)
head_label.place(relwidth=1)

line = Label(base, width=150, bg=BG_GRAY)


#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
ChatLog.focus()

#Create Button to send message
SendButton = Button(base, font=("Verdana", 12,'bold'), text="Send", width="12", height=15,
                    bd=0, bg="#ed9061", activebackground="#3c9d9b",fg='#ffffff',
                    command=lambda: send)

#Create the box to enter message
EntryBox = Text(base, bg="white",width="29", height="5", font="Arial", background="#dddddd")
EntryBox.focus()
EntryBox.bind("<Return>", send)
#EntryBox.bind("<Return>", send)

# bottom label
# bottom_label = Label(base, bg=BG_GRAY, height=80)
# bottom_label.place(relwidth=1, rely=0.825)
        
# message entry box
#EntryBox = Entry(bottom_label, fg=TEXT_COLOR, font=FONT)
# msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, |relx=0.011)
# msg_entry.focus()


#Place all components on the screen
# scrollbar.place(relheight=1, relx=0.974)
# ChatLog.place(relheight=1, width=1)
# EntryBox.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
# SendButton.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.20)\

scrollbar.place(x=475,y=6, height=500)
line.place(x=0,y=35, height=1, width=470)
ChatLog.place(x=5,y=40, height=400, width=470)
EntryBox.place(x=0, y=440, height=60, width=300)
SendButton.place(x=300, y=440, height=60, width=175)

In [54]:
base.mainloop()

1/1 [==============================] - 0s 24ms/step
